In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re

In [2]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_influencer"
api_name = "get_author_base_info"

In [3]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".json", "").split("_")[-1]),
} for blob in storage_client.list_blobs("0_raw_data",prefix="2_xingtu/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-11/get_author_base_info_0911_0.json, 1726037771767688>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-17/get_author_base_info_0917_0.json, 1726565274658091>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-18/get_author_base_info_0918_0.json, 1726633853328119>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-19/get_author_base_info_0919_0.json, 1726719805616242>,
  'date': datetime.datetime(2024, 9, 19, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-09/get_author_base_info_241009_0.json, 1728452332203839>,
  'date': datetime.datetime(2024, 10, 9, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_base_info_1011_0.json, 1728632483067101>,
  'date': datetime.datetime(2024, 10

In [4]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs("3_staging_area",prefix="1_xingtu/") if table_name == blob.name.split("/")[1] and api_name in blob.name and blob.name.split("/")[2] and blob.name.split("/")[3]]
processed_blobs

[{'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-11/get_author_base_info_240920_0.parquet, 1726803504667584>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-17/get_author_base_info_240920_0.parquet, 1726803627188174>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-18/get_author_base_info_240920_0.parquet, 1726803629892657>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-09-19/get_author_base_info_240920_0.parquet, 1726803631833151>,
  'date': datetime.datetime(2024, 9, 19, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_influencer/2024-10-09/get_author_base_info_241009_0.parquet, 1728457802828707>,
  'date': datetime.datetime(2024, 10, 9, 0, 0),
  'batch_number': 0}]

In [5]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 13):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch_number': 0,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_base_info_1011_0.json, 1728632483067101>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)}]
1


In [6]:
def get_author_base_info_data(raw_data):
    result = []
    for data in raw_data:
        get_author_base_info = data.get("get_author_base_info")
        if get_author_base_info:
            record = {}
            profile_id = data["profile_id"]
            record["user_id"] = profile_id
            record["avg_play"] = get_author_base_info["avg_play"]
            record["city"] = get_author_base_info["city"]
            record["core_user_id"] = get_author_base_info["core_user_id"]
            record["e_commerce_enable"] = get_author_base_info["e_commerce_enable"]
            record["is_star"] = get_author_base_info["is_star"]
            record["lowest_price"] = get_author_base_info["lowest_price"]
            record["province"] = get_author_base_info["province"]
            record["short_id"] = get_author_base_info["short_id"]
            record["tags_ids"] = get_author_base_info["tags_ids"]
            record["tags_ids_level_two"] = get_author_base_info["tags_ids_level_two"]
            record["unique_id"] = get_author_base_info["unique_id"]
            record["nickname"] = get_author_base_info["nick_name"]
            record["image_url"] = get_author_base_info["avatar_uri"]
            record["fans_count"] = get_author_base_info["follower"]
            record["sex"] = get_author_base_info["gender"]
            record["create_time"] = datetime.fromtimestamp(int(get_author_base_info["create_time"])).replace(tzinfo=timezone.utc)
            record["modify_time"] = datetime.fromtimestamp(int(get_author_base_info["modify_time"])).replace(tzinfo=timezone.utc)
            result.append(record)
    return result

In [7]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    author_base_info_data = get_author_base_info_data(raw_data)
    df = pd.DataFrame(author_base_info_data)
    df.to_parquet("gs://3_staging_area/1_xingtu/douyin_influencer/" + str(item["date"].date()) + "/get_author_base_info_" + today.strftime("%y%m%d") + "_" + str(item["batch_number"]) + ".parquet")